## 참고 자료\n\n- [Biopython Tutorial and Cookbook](http://biopython.org/DIST/docs/tutorial/Tutorial.html) - Chapter 9: Accessing NCBI's Entrez databases\n- [NCBI E-utilities Documentation](https://www.ncbi.nlm.nih.gov/books/NBK25497/)\n- [Biopython Entrez Module Documentation](https://biopython.org/docs/latest/api/Bio.Entrez.html)\n- [NCBI API 사용 가이드라인](https://www.ncbi.nlm.nih.gov/home/api/)\n- [PMC XML Schema](https://jats.nlm.nih.gov/) - JATS (Journal Article Tag Suite)"

## 모범 사례 및 팁\n\n### Biopython Entrez 사용 시 주의사항\n\n1. **이메일 주소 설정**: 반드시 `Entrez.email`을 설정해야 합니다\n2. **API 키 사용**: 더 높은 요청 제한을 위해 API 키를 설정하세요\n3. **적절한 지연**: API 호출 간 적절한 지연 시간을 유지하세요\n4. **에러 처리**: XML 파싱 및 네트워크 오류에 대한 예외 처리를 포함하세요\n5. **핸들 관리**: 사용 후 반드시 핸들을 닫으세요\n\n### 대용량 데이터 처리 팁\n\n- `usehistory=y`를 사용하여 History 서버 활용\n- 배치 처리로 여러 ID를 한 번에 가져오기\n- `Entrez.parse()`를 사용하여 메모리 효율적인 파싱\n- 중간 결과를 파일로 저장하여 재시작 가능한 프로세스 구축"

In [ ]:
# 데이터셋 상세 분석\nif len(df_pmc) > 0:\n    print(\"=== 데이터셋 상세 분석 ===\")\n    \n    # 기본 통계\n    print(f\"총 논문 수: {len(df_pmc)}\")\n    print(f\"유니크 저널 수: {df_pmc['journal'].nunique()}\")\n    print(f\"평균 제목 길이: {df_pmc['title'].str.len().mean():.1f} 문자\")\n    print(f\"평균 초록 길이: {df_pmc['abstract'].str.len().mean():.1f} 문자\")\n    print(f\"평균 전체 텍스트 길이: {df_pmc['full_text'].str.len().mean():.1f} 문자\")\n    \n    # 연도별 분포 (간단한 분석)\n    years = df_pmc['pubdate'].str.extract(r'(\\d{4})')[0]\n    if not years.empty:\n        print(f\"\\n=== 발행 연도 분포 ===\")\n        year_counts = years.value_counts().sort_index()\n        for year, count in year_counts.items():\n            print(f\"{year}: {count}편\")\n    \n    # 저널별 분포\n    if df_pmc['journal'].nunique() > 1:\n        print(f\"\\n=== 저널별 분포 ===\")\n        journal_counts = df_pmc['journal'].value_counts()\n        for journal, count in journal_counts.items():\n            print(f\"{journal}: {count}편\")\n    \n    # 첫 번째 논문의 전체 텍스트 샘플 보기\n    if len(df_pmc) > 0 and len(df_pmc.iloc[0]['full_text']) > 0:\n        print(f\"\\n=== 전체 텍스트 샘플 (첫 1000자) ===\")\n        sample_text = df_pmc.iloc[0]['full_text'][:1000]\n        print(sample_text)\n        print(\"...\")\n    \n    # 데이터 품질 확인\n    print(f\"\\n=== 데이터 품질 ===\")\n    print(f\"제목이 있는 논문: {(df_pmc['title'].str.len() > 0).sum()}/{len(df_pmc)}\")\n    print(f\"초록이 있는 논문: {(df_pmc['abstract'].str.len() > 0).sum()}/{len(df_pmc)}\")\n    print(f\"전체 텍스트가 있는 논문: {(df_pmc['full_text'].str.len() > 0).sum()}/{len(df_pmc)}\")\n    print(f\"저자 정보가 있는 논문: {(df_pmc['authors'].str.len() > 0).sum()}/{len(df_pmc)}\")\n    \nelse:\n    print(\"분석할 데이터가 없습니다.\")"

## 데이터 탐색 및 분석"

In [ ]:
import pandas as pd\nimport time\nfrom collections import defaultdict\n\ndef extract_text_recursive(element):\n    \"\"\"XML 요소에서 텍스트를 재귀적으로 추출합니다.\"\"\"\n    if isinstance(element, str):\n        return element.strip()\n    elif isinstance(element, dict):\n        text_parts = []\n        # 직접 텍스트\n        if '#text' in element:\n            text_parts.append(element['#text'].strip())\n        \n        # 다른 텍스트 요소들\n        for key, value in element.items():\n            if key not in ['@id', '@rid', '@ref-type', '#text'] and value:\n                if isinstance(value, (dict, list)):\n                    text_parts.append(extract_text_recursive(value))\n                elif isinstance(value, str):\n                    text_parts.append(value.strip())\n        \n        return ' '.join(filter(None, text_parts))\n    elif isinstance(element, list):\n        return ' '.join(extract_text_recursive(item) for item in element)\n    else:\n        return str(element).strip() if element else \"\"\n\ndef extract_full_text_from_sections(sections):\n    \"\"\"PMC 논문 섹션에서 전체 텍스트를 추출합니다.\"\"\"\n    if not sections:\n        return \"\"\n    \n    if not isinstance(sections, list):\n        sections = [sections]\n    \n    full_text_parts = []\n    \n    for section in sections:\n        section_text = extract_text_recursive(section)\n        if section_text:\n            full_text_parts.append(section_text)\n    \n    return '\\n\\n'.join(full_text_parts)\n\ndef parse_pmc_article_biopython(record):\n    \"\"\"Biopython으로 파싱된 PMC 레코드에서 정보를 추출합니다.\"\"\"\n    try:\n        article = record.get('article', {})\n        \n        # 메타데이터 추출\n        front = article.get('front', {})\n        article_meta = front.get('article-meta', {})\n        \n        # 제목\n        title_group = article_meta.get('title-group', {})\n        title = title_group.get('article-title', '')\n        if isinstance(title, dict):\n            title = extract_text_recursive(title)\n        \n        # PMCID\n        pmcid = \"\"\n        article_ids = article_meta.get('article-id', [])\n        if not isinstance(article_ids, list):\n            article_ids = [article_ids]\n        \n        for aid in article_ids:\n            if isinstance(aid, dict) and aid.get('@pub-id-type') == 'pmcid':\n                pmcid = aid.get('#text', '')\n                break\n        \n        # 저널 정보\n        journal_meta = front.get('journal-meta', {})\n        journal_title_group = journal_meta.get('journal-title-group', {})\n        journal_title = journal_title_group.get('journal-title', '')\n        if isinstance(journal_title, dict):\n            journal_title = extract_text_recursive(journal_title)\n        \n        # 발행 날짜\n        pub_date = \"\"\n        pub_date_info = article_meta.get('pub-date', {})\n        if isinstance(pub_date_info, list):\n            pub_date_info = pub_date_info[0]\n        if isinstance(pub_date_info, dict):\n            year = pub_date_info.get('year', '')\n            month = pub_date_info.get('month', '')\n            day = pub_date_info.get('day', '')\n            pub_date = '-'.join(filter(None, [str(year), str(month), str(day)]))\n        \n        # 저자들\n        authors = []\n        contrib_group = article_meta.get('contrib-group', {})\n        contribs = contrib_group.get('contrib', [])\n        if not isinstance(contribs, list):\n            contribs = [contribs]\n        \n        for contrib in contribs:\n            if isinstance(contrib, dict) and contrib.get('@contrib-type') == 'author':\n                name_info = contrib.get('name', {})\n                if isinstance(name_info, dict):\n                    given_names = name_info.get('given-names', '')\n                    surname = name_info.get('surname', '')\n                    full_name = f\"{given_names} {surname}\".strip()\n                    if full_name:\n                        authors.append(full_name)\n        \n        # 초록\n        abstract_text = \"\"\n        abstract_info = article_meta.get('abstract', {})\n        if isinstance(abstract_info, dict):\n            abstract_text = extract_text_recursive(abstract_info)\n        \n        # 전체 텍스트\n        full_text = \"\"\n        body = article.get('body', {})\n        if 'sec' in body:\n            full_text = extract_full_text_from_sections(body['sec'])\n        \n        return {\n            'pmcid': pmcid,\n            'title': title,\n            'journal': journal_title,\n            'pubdate': pub_date,\n            'authors': ', '.join(authors),\n            'abstract': abstract_text,\n            'full_text': full_text\n        }\n    \n    except Exception as e:\n        print(f\"레코드 파싱 중 오류 발생: {e}\")\n        return None\n\ndef fetch_pmc_articles_biopython(search_term, max_articles=5, delay=0.5):\n    \"\"\"Biopython을 사용하여 PMC에서 논문들을 가져와 데이터셋을 생성합니다.\"\"\"\n    print(f\"검색어: {search_term}\")\n    \n    # 1. PMC에서 검색\n    handle = Entrez.esearch(db=\"pmc\", term=search_term, retmax=max_articles)\n    search_results = Entrez.read(handle)\n    handle.close()\n    \n    pmc_ids = search_results['IdList']\n    print(f\"찾은 논문 수: {len(pmc_ids)}\")\n    \n    if not pmc_ids:\n        return pd.DataFrame()\n    \n    # 2. 각 논문의 상세 정보 가져오기\n    articles_data = []\n    \n    for i, pmcid in enumerate(pmc_ids):\n        print(f\"처리 중: {i+1}/{len(pmc_ids)} - PMC{pmcid}\")\n        \n        try:\n            # PMC 전체 텍스트 가져오기\n            handle = Entrez.efetch(db=\"pmc\", id=pmcid, rettype=\"full\", retmode=\"xml\")\n            records = Entrez.parse(handle)\n            \n            for record in records:\n                article_data = parse_pmc_article_biopython(record)\n                if article_data:\n                    # PMCID가 비어있으면 검색 결과의 ID 사용\n                    if not article_data['pmcid']:\n                        article_data['pmcid'] = f\"PMC{pmcid}\"\n                    articles_data.append(article_data)\n                break  # 첫 번째 레코드만 처리\n            \n            handle.close()\n            \n        except Exception as e:\n            print(f\"PMC{pmcid} 처리 중 오류 발생: {e}\")\n            continue\n        \n        # API 제한을 위한 지연\n        time.sleep(delay)\n    \n    # 3. DataFrame 생성\n    df = pd.DataFrame(articles_data)\n    return df\n\n# 예제: LLM과 생물정보학 관련 논문들로 데이터셋 생성\nsearch_query = '(\"LLM\" OR \"large language model\")[Title/Abstract] AND bioinformatics'\nprint(\"PMC에서 데이터셋 생성 중...\")\ndf_pmc = fetch_pmc_articles_biopython(search_query, max_articles=3, delay=0.5)\n\nprint(f\"\\n=== 데이터셋 요약 ===\")\nprint(f\"총 논문 수: {len(df_pmc)}\")\nif len(df_pmc) > 0:\n    print(f\"컬럼: {list(df_pmc.columns)}\")\n    print(f\"전체 텍스트가 있는 논문: {(df_pmc['full_text'].str.len() > 0).sum()}\")\n    print(f\"초록이 있는 논문: {(df_pmc['abstract'].str.len() > 0).sum()}\")\n    \n    # 샘플 논문 정보 표시\n    if len(df_pmc) > 0:\n        sample = df_pmc.iloc[0]\n        print(f\"\\n=== 샘플 논문 정보 ===\")\n        print(f\"PMCID: {sample['pmcid']}\")\n        print(f\"제목: {sample['title'][:100]}...\")\n        print(f\"저널: {sample['journal']}\")\n        print(f\"저자: {sample['authors'][:100]}...\")\n        print(f\"전체 텍스트 길이: {len(sample['full_text'])} 문자\")\n\ndf_pmc.head()"

## 데이터셋 생성: PMC에서 ML용 데이터 추출\n\nBiopython을 사용하여 PMC에서 전체 텍스트를 포함한 데이터셋을 생성해보겠습니다."

In [ ]:
# PMC 논문의 전체 텍스트 가져오기\nif pmc_ids:\n    first_pmcid = pmc_ids[0]\n    \n    handle = Entrez.efetch(db=\"pmc\", id=first_pmcid, rettype=\"full\", retmode=\"xml\")\n    \n    # PMC XML은 크기가 클 수 있으므로 Entrez.parse() 사용\n    try:\n        records = Entrez.parse(handle)\n        \n        for record in records:\n            # PMC XML 구조 탐색\n            article = record.get('article', {})\n            \n            # 메타데이터 추출\n            front = article.get('front', {})\n            article_meta = front.get('article-meta', {})\n            \n            # 제목\n            title_group = article_meta.get('title-group', {})\n            title = title_group.get('article-title', 'N/A')\n            if isinstance(title, dict):\n                title = title.get('#text', str(title))\n            \n            print(f\"PMCID: {first_pmcid}\")\n            print(f\"제목: {title}\")\n            \n            # 저널 정보\n            journal_meta = front.get('journal-meta', {})\n            journal_title_group = journal_meta.get('journal-title-group', {})\n            journal_title = journal_title_group.get('journal-title', 'N/A')\n            if isinstance(journal_title, dict):\n                journal_title = journal_title.get('#text', str(journal_title))\n            print(f\"저널: {journal_title}\")\n            \n            # 본문에서 섹션 정보 가져오기\n            body = article.get('body', {})\n            if 'sec' in body:\n                sections = body['sec']\n                if not isinstance(sections, list):\n                    sections = [sections]\n                \n                print(f\"\\n=== 논문 섹션 ({len(sections)}개) ===\")\n                for i, section in enumerate(sections[:3]):  # 처음 3개 섹션만 표시\n                    section_title = section.get('title', f'Section {i+1}')\n                    print(f\"{i+1}. {section_title}\")\n                    \n                    # 첫 번째 문단의 일부 내용 표시\n                    if 'p' in section:\n                        paragraphs = section['p']\n                        if not isinstance(paragraphs, list):\n                            paragraphs = [paragraphs]\n                        \n                        for p in paragraphs[:1]:  # 첫 번째 문단만\n                            if isinstance(p, str):\n                                text = p[:200] + \"...\" if len(p) > 200 else p\n                                print(f\"   {text}\")\n                            elif isinstance(p, dict) and '#text' in p:\n                                text = p['#text'][:200] + \"...\" if len(p['#text']) > 200 else p['#text']\n                                print(f\"   {text}\")\n                    print()\n            \n            break  # 첫 번째 레코드만 처리\n    \n    except Exception as e:\n        print(f\"PMC 데이터 파싱 중 오류 발생: {e}\")\n        print(\"원시 XML 데이터의 일부를 확인해보겠습니다...\")\n        \n        # 원시 XML의 처음 부분 표시\n        handle = Entrez.efetch(db=\"pmc\", id=first_pmcid, rettype=\"full\", retmode=\"xml\")\n        xml_data = handle.read()\n        print(xml_data[:1000].decode('utf-8'))\n        handle.close()\n    \n    handle.close()\nelse:\n    print(\"검색된 PMC ID가 없습니다.\")"

### PMC 논문의 전체 텍스트 가져오기"

In [ ]:
# PubMed 논문의 상세 정보 가져오기\nif pubmed_ids:\n    first_pmid = pubmed_ids[0]\n    \n    handle = Entrez.efetch(db=\"pubmed\", id=first_pmid, rettype=\"abstract\", retmode=\"xml\")\n    records = Entrez.parse(handle)\n    \n    for record in records:\n        article = record['MedlineCitation']['Article']\n        \n        print(f\"PMID: {first_pmid}\")\n        print(f\"제목: {article.get('ArticleTitle', 'N/A')}\")\n        \n        # 저자 정보\n        if 'AuthorList' in article:\n            authors = []\n            for author in article['AuthorList']:\n                if 'LastName' in author and 'ForeName' in author:\n                    authors.append(f\"{author['ForeName']} {author['LastName']}\")\n            print(f\"저자: {', '.join(authors[:5])}{'...' if len(authors) > 5 else ''}\")\n        \n        # 저널 정보\n        journal = article.get('Journal', {})\n        if journal:\n            print(f\"저널: {journal.get('Title', 'N/A')}\")\n            if 'JournalIssue' in journal:\n                issue = journal['JournalIssue']\n                pub_date = issue.get('PubDate', {})\n                year = pub_date.get('Year', 'N/A')\n                print(f\"발행년도: {year}\")\n        \n        # 초록\n        if 'Abstract' in article:\n            abstract = article['Abstract']\n            if 'AbstractText' in abstract:\n                abstract_text = abstract['AbstractText']\n                if isinstance(abstract_text, list):\n                    # 구조화된 초록\n                    print(\"\\n=== 초록 ===\")\n                    for section in abstract_text:\n                        if isinstance(section, dict) and '@Label' in section:\n                            print(f\"**{section['@Label']}**: {section.get('#text', section)}\")\n                        else:\n                            print(section)\n                else:\n                    print(f\"\\n=== 초록 ===\\n{abstract_text}\")\n        \n        break  # 첫 번째 레코드만 처리\n    \n    handle.close()\nelse:\n    print(\"검색된 PubMed ID가 없습니다.\")"

### PubMed 논문의 상세 정보 (초록 및 메타데이터)"

## EFetch: UID로 상세 레코드 가져오기\n\n`Entrez.efetch()`를 사용하여 특정 ID의 전체 레코드를 가져올 수 있습니다."

In [ ]:
# PMC에서 LLM과 생물정보학 관련 논문 검색\nquery = '\"LLM\"[Title] AND bioinformatics'\n\nhandle = Entrez.esearch(db=\"pmc\", term=query, retmax=10)\nrecord = Entrez.read(handle)\nhandle.close()\n\nprint(f\"검색어: {query}\")\nprint(f\"총 결과 수: {record['Count']}\")\nprint(f\"가져온 ID 수: {len(record['IdList'])}\")\nprint(f\"쿼리 번역: {record['QueryTranslation']}\")\n\nprint(\"\\n=== PMC ID 목록 ===\")\npmc_ids = record['IdList']\nfor i, pmcid in enumerate(pmc_ids):\n    print(f\"{i+1}. PMCID: {pmcid}\")"

### PMC에서 LLM 및 생물정보학 논문 검색"

In [ ]:
# PubMed에서 LLM과 생물정보학 관련 논문 검색\nquery = '\"LLM\"[Title] AND bioinformatics'\n\nhandle = Entrez.esearch(db=\"pubmed\", term=query, retmax=10)\nrecord = Entrez.read(handle)\nhandle.close()\n\nprint(f\"검색어: {query}\")\nprint(f\"총 결과 수: {record['Count']}\")\nprint(f\"가져온 ID 수: {len(record['IdList'])}\")\nprint(f\"쿼리 번역: {record['QueryTranslation']}\")\n\nprint(\"\\n=== PubMed ID 목록 ===\")\npubmed_ids = record['IdList']\nfor i, pmid in enumerate(pubmed_ids):\n    print(f\"{i+1}. PMID: {pmid}\")"

### PubMed에서 LLM 및 생물정보학 논문 검색"

## ESearch: 검색어로 UID 목록 조회\n\n`Entrez.esearch()`를 사용하여 다양한 데이터베이스에서 검색을 수행할 수 있습니다."

In [ ]:
# ClinVar 데이터베이스 정보 가져오기\nhandle = Entrez.einfo(db=\"clinvar\")\nrecord = Entrez.read(handle)\nhandle.close()\n\ndb_info = record[\"DbInfo\"]\n\nprint(f\"데이터베이스: {db_info['DbName']}\")\nprint(f\"설명: {db_info['Description']}\")\nprint(f\"마지막 업데이트: {db_info['LastUpdate']}\")\nprint(f\"레코드 수: {db_info['Count']}\")\n\nprint(\"\\n=== 검색 필드 (처음 15개) ===\")\nfor field in db_info['FieldList'][:15]:\n    print(f\"{field['Name']}: {field['FullName']} (용어 수: {field['TermCount']})\")\n\nprint(f\"\\n총 검색 필드 수: {len(db_info['FieldList'])}\")\nprint(f\"총 연결 데이터베이스 수: {len(db_info['LinkList'])}\")"

### ClinVar 데이터베이스 정보"

In [ ]:
# PubMed 데이터베이스 정보 가져오기\nhandle = Entrez.einfo(db=\"pubmed\")\nrecord = Entrez.read(handle)\nhandle.close()\n\ndb_info = record[\"DbInfo\"]\n\nprint(f\"데이터베이스: {db_info['DbName']}\")\nprint(f\"설명: {db_info['Description']}\")\nprint(f\"마지막 업데이트: {db_info['LastUpdate']}\")\nprint(f\"레코드 수: {db_info['Count']}\")\n\nprint(\"\\n=== 검색 필드 (처음 10개) ===\")\nfor field in db_info['FieldList'][:10]:\n    print(f\"{field['Name']}: {field['FullName']} (용어 수: {field['TermCount']})\")\n\nprint(\"\\n=== 연결된 데이터베이스 (처음 10개) ===\")\nfor link in db_info['LinkList'][:10]:\n    print(f\"{link['Name']}: {link['DbTo']} 데이터베이스로 연결\")"

### PubMed 데이터베이스 정보"

## EInfo: 데이터베이스 정보 확인\n\n`Entrez.einfo()`를 사용하여 NCBI 데이터베이스의 메타데이터를 확인할 수 있습니다."

In [ ]:
# Biopython Entrez 설정\nfrom Bio import Entrez\nimport os\nfrom dotenv import load_dotenv\n\nload_dotenv()\n\n# 이메일 설정 (필수)\nEntrez.email = os.getenv(\"EUTILS_EMAIL\", \"student@example.com\")\n\n# API 키 설정 (선택사항, 더 높은 요청 제한)\napi_key = os.getenv(\"EUTILS_API_KEY\")\nif api_key:\n    Entrez.api_key = api_key\n\n# 도구 이름 설정\nEntrez.tool = os.getenv(\"EUTILS_TOOL\", \"biopython-class-notebook\")\n\nprint(f\"Email: {Entrez.email}\")\nprint(f\"Tool: {Entrez.tool}\")\nprint(f\"API Key set: {bool(api_key)}\")"

### 기본 설정\n\nNCBI는 모든 사용자에게 이메일 주소를 제공하도록 요구합니다."

### 패키지 설치\n\nBiopython을 설치해야 합니다.\n\n```bash\npip install biopython pandas\n```"

### 핵심 개념\n\n* **Bio.Entrez**: Biopython에서 제공하는 NCBI Entrez 데이터베이스 접근 모듈\n* **내장 파싱**: XML 응답을 자동으로 파싱하여 Python 딕셔너리로 변환\n* **자동 지연**: API 호출 간 적절한 지연 시간을 자동으로 관리\n* **에러 처리**: HTTP 및 XML 파싱 에러를 적절히 처리\n* **이메일 요구사항**: NCBI 정책에 따라 이메일 주소를 필수로 설정\n\n### Biopython Entrez의 주요 장점\n\n* **간단한 사용법**: `requests` 대비 더 간단한 API\n* **자동 파싱**: XML을 딕셔너리로 자동 변환\n* **내장 검증**: 잘못된 파라미터 검증\n* **역사 서버 지원**: WebEnv와 query_key 자동 관리\n* **배치 처리**: 대용량 데이터 처리를 위한 최적화\n\n### 주요 함수들\n\n* `Entrez.einfo()` — 데이터베이스 정보 확인\n* `Entrez.esearch()` — 검색어로 UID 목록 조회  \n* `Entrez.esummary()` — UID 요약 정보 반환\n* `Entrez.efetch()` — UID의 전체 레코드 반환\n* `Entrez.elink()` — 데이터베이스 간 연관 링크 조회\n* `Entrez.epost()` — UID 목록을 History 서버에 업로드\n* `Entrez.parse()` — XML 결과를 파싱하여 Python 객체로 변환\n* `Entrez.read()` — 단일 레코드를 파싱하여 딕셔너리로 변환"

# Biopython Entrez 모듈 실습

**학습목표**: Biopython의 `Bio.Entrez` 모듈을 사용해 `NCBI Entrez E-utilities`를 호출하고 생물의학 데이터를 효율적으로 처리하는 방법을 배운다.

**Last updated**: 2025-11-03

**Reference**: *Biopython Tutorial and Cookbook* (Chapter 9: Accessing NCBI's Entrez databases)